In [1]:
from grafos import Accion
from grafos import Estado
from grafos import Nodo
from grafos import Problema

Construir la función de ejecución del problema

In [2]:
def voraz(problema):
    raiz = crea_nodo_raiz(problema)
    frontera = [raiz, ]
    explorados = set()
    while True:
        if not frontera:
            return None
        nodo = sacar_siguiente(frontera, 'heuristica',
                               objetivos=problema.estado_objetivo)
        if problema.es_objetivo(nodo.estado):
            return nodo
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crea_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if hijo.estado in explorados or hijo.estado in estados_frontera:
                buscar = [nodo for nodo in frontera
                          if nodo.estado == hijo.estado]
                if buscar:
                    heuristic_hijo = [hijo.heuristicas[objetivo.nombre]
                                      for objetivo
                                      in problema.estado_objetivo]
                    heuristic_buscar = [buscar[0].heuristicas[objetivo.nombre]
                                        for objetivo
                                        in problema.estado_objetivo]
                    minimo_hijo = min(heuristic_hijo)
                    minimo_buscar = min(heuristic_buscar)
                    if minimo_hijo < minimo_buscar:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            else:
                frontera.append(hijo)               

Métodos auxiliares

In [3]:
def crea_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    raiz.costo = 0
    raiz.heuristicas = problema.heuristicas[estado_raiz.nombre]
    raiz.valores = dict(raiz.heuristicas.items())
    return raiz

def crea_nodo_hijo(problema, padre, accion, agregar = True):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    hijo.heuristicas = problema.heuristicas[hijo.estado.nombre]
    hijo.valores = {estado: heuristica + hijo.costo for estado, heuristica in hijo.heuristicas.items()}
    if agregar:
        hijo.padre = padre
        padre.hijos.append(hijo)
    return hijo

def sacar_siguiente(frontera, metrica = 'valor', criterio = 'menor', objetivos = None):
    mejor = frontera[0]
    for nodo in frontera[1:]:
        for objetivo in objetivos:
            if metrica == 'valor':
                valor_nodo = nodo.valores[objetivo.nombre]
                valor_mejor = mejor.valores[objetivo.nombre]
                if(criterio == 'menor' and
                   valor_nodo < valor_mejor):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     valor_nodo > valor_mejor):
                    mejor = nodo
            elif metrica == 'heuristica':
                heuristica_nodo = nodo.heuristicas[objetivo.nombre]
                heuristica_mejor = mejor.heuristicas[objetivo.nombre]
                if(criterio == 'menor' and
                   heuristica_nodo < heuristica_mejor):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     heuristica_nodo > heuristica_mejor):
                    mejor = nodo
            elif metrica == 'costo':
                if(criterio == 'menor' and
                   nodo.costo_camino < mejor.costo_camino):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     nodo.costo_camino > mejor.costo_camino):
                    mejor = nodo
    frontera.remove(mejor)
    return mejor

def muestra_solucion(objetivo = None):
    if not objetivo:
        print("No hay solución")
        return
    nodo = objetivo
    while nodo:
        msg = "Estado {0}, Valor {1}"
        estado = nodo.estado.nombre
        valores = [nodo.valores[objetivo.nombre]
                   for objetivo
                   in problema_resolver.estado_objetivo]
        valor = min(valores)
        print(msg.format(estado, valor))
        msg = "  Costo: {0}"
        costo_total = nodo.costo
        print(msg.format(costo_total))
        msg = "  Heurística: {0}"
        heuristicas_objetivos = [nodo.heuristicas[objetivo.nombre]
                                 for objetivo
                                 in problema_resolver.estado_objetivo]
        heuristica = min(heuristicas_objetivos)
        print(msg.format(heuristica))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            if accion:
                msg = "<--- {0} [{1}] ---"
                print(msg.format(accion, costo))
        nodo = nodo.padre

Crear la estructura de datos

In [4]:
from Datos import Datos

if __name__ == '__main__':
    datos = Datos()
    
    # objetivo_1 = [kosos]
    # problema_1 = Problema(lanoi, objetivo_1, acciones, costos, heuristicas)

    objetivo_2 = [datos.tolu]
    problema_2 = Problema(
        datos.manizales, 
        objetivo_2, 
        datos.acciones, 
        datos.costos
    )


In [5]:
problema_resolver = problema_2
solucion = voraz(problema_resolver)
muestra_solucion(solucion)

KeyError: 'Tolu'